In [4]:
import shutil
from pathlib import Path

folder = Path("C:/Users/ojass/Documents/scpr/Leaves/Dataset/dataset_split")
if folder.exists():
    shutil.rmtree(folder)


In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Paths
base_dir = Path("C:/Users/ojass/Documents/scpr/Leaves")

image_dir = base_dir
output_dir = base_dir / "sorted_dataset"
output_dir.mkdir(parents=True, exist_ok=True)

# Load metadata
df = pd.read_csv(base_dir / "flavia_metadata.csv")

# Process each row to move images into species folders
for _, row in tqdm(df.iterrows(), total=len(df)):
    species_name = row["Scientific Name"].split(" ")[0]  # Shorten name for folder
    image_range = row["filename"]
    
    if '-' not in image_range:
        continue
    
    start, end = map(int, image_range.split('-'))
    species_dir = output_dir / species_name
    species_dir.mkdir(parents=True, exist_ok=True)
    
    for img_id in range(start, end + 1):
        for ext in [".jpg", ".png"]:
            src = image_dir / f"{img_id}{ext}"
            dst = species_dir / f"{img_id}{ext}"
            if src.exists():
                shutil.copy(src, dst)
                break


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:06<00:00,  4.88it/s]


In [10]:
import os
import shutil
import random
from pathlib import Path
from tqdm import tqdm

# Settings
source_root = Path("C:/Users/ojass/Documents/scpr/Leaves/sorted_dataset")
train_root = source_root / "train"
val_root = source_root / "val"
split_ratio = 0.8  # 80% train, 20% val

# Make sure output folders are clean
if train_root.exists():
    shutil.rmtree(train_root)
if val_root.exists():
    shutil.rmtree(val_root)
train_root.mkdir(parents=True)
val_root.mkdir(parents=True)

# Iterate through species folders
for species_dir in tqdm([d for d in source_root.iterdir() if d.is_dir() and d.name not in ["train", "val"]]):
    images = list(species_dir.glob("*.[jp][pn]g"))  # jpg or png
    random.shuffle(images)

    split_idx = int(len(images) * split_ratio)
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    # Create subfolders
    train_sub = train_root / species_dir.name
    val_sub = val_root / species_dir.name
    train_sub.mkdir(parents=True, exist_ok=True)
    val_sub.mkdir(parents=True, exist_ok=True)

    # Copy
    for img in train_images:
        shutil.copy(img, train_sub / img.name)
    for img in val_images:
        shutil.copy(img, val_sub / img.name)

print("✅ Train/Val split done.")


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.52it/s]

✅ Train/Val split done.


In [11]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from ultralytics import YOLO

# Paths to your dataset
DATASET_PATH = "C:/Users/ojass/Documents/scpr/Leaves/sorted_dataset"

# Choose a model variant: yolov8n-cls.pt, yolov8s-cls.pt, yolov8m-cls.pt, yolov8l-cls.pt, yolov8x-cls.pt
MODEL = "yolov8n-cls.pt"

# Train the classification model
model = YOLO(MODEL)
model.train(
    data=DATASET_PATH,
    epochs=10,
    imgsz=224,      # You can change this depending on image resolution
    batch=32,       # Adjust based on your GPU
    project="leaf_classification_yolo",
    name="yolov8n_cls",
    verbose=True
)


C:\Users\ojass\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.100  Python-3.12.1 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=C:/Users/ojass/Documents/scpr/Leaves/sorted_dataset, epochs=10, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=leaf_classification_yolo, name=yolov8n_cls2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic

train: Scanning C:\Users\ojass\Documents\scpr\Leaves\sorted_dataset\train... 1515 images, 0 corrupt: 100%|██████████| 1
val: Scanning C:\Users\ojass\Documents\scpr\Leaves\sorted_dataset\val... 392 images, 0 corrupt: 100%|██████████| 392/39


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000303, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to leaf_classification_yolo\yolov8n_cls2
Starting training for 10 epochs...


In [1]:
import torch
torch.cuda.empty_cache()


C:\Users\ojass\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolov8n.pt")  

model.train(
    data="leaves.yaml", 
    epochs=25, 
    imgsz=640,
    mosaic=1,  # Increase synthetic data mixing
    mixup=0.8,  # Stronger augmentation
    dropout=0.3  # Prevent overfitting
)



New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.100  Python-3.12.1 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=leaves.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train38, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

train: Scanning C:\Users\ojass\Documents\scpr\Leaves\Dataset\train\labels.cache... 1514 images, 0 backgrounds, 0 corrup
val: Scanning C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\labels.cache... 393 images, 0 backgrounds, 0 corrupt: 10


Plotting labels to runs\detect\train38\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00027, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train38
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25      2.03G        1.7      3.725      2.224         57        640: 100%|██████████| 95/95 [00:31<00:00,  3.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05

                   all        393        393      0.797      0.807      0.879      0.497


...

                   all        393        393          1          1      0.995      0.995



25 epochs completed in 0.226 hours.
Optimizer stripped from runs\detect\train38\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train38\weights\best.pt, 6.2MB

Validating runs\detect\train38\weights\best.pt...
Ultralytics 8.3.100  Python-3.12.1 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 72 layers, 3,012,083 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05


                   all        393        393          1          1      0.995      0.995
        Tonna sinensis        393        393          1          1      0.995      0.995
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs\detect\train38


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([23])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000026ECA203170>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048

In [3]:
results = model.val()

# Print validation results
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")

# Precision and Recall (taking the mean across all classes)
precision = results.box.p.mean()
recall = results.box.r.mean()

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Ultralytics 8.3.100  Python-3.12.1 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 72 layers, 3,012,083 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\labels.cache... 393 images, 0 backgrounds, 0 corrupt: 10
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:07


                   all        393        393          1          1      0.995      0.995
        Tonna sinensis        393        393          1          1      0.995      0.995
Speed: 0.7ms preprocess, 4.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs\detect\train382
mAP50: 0.9950
mAP50-95: 0.9950
Precision: 0.9999
Recall: 1.0000


In [4]:
from ultralytics import YOLO

# Load your trained model (use the correct path)
model = YOLO('runs/detect/train38/weights/best.pt')


In [28]:
import numpy as np

cache_data = np.load(r'C:\Users\ojass\Documents\scpr\Leaves\Dataset\train\labels.cache', allow_pickle=True).item()
print(type(cache_data))  # should be dict
print(cache_data.keys())  # see what's inside
from collections import Counter

labels = [int(v['cls']) if isinstance(v['cls'], (int, float)) else int(v['cls'][0]) for v in cache_data['labels']]
 # Adjust if needed based on keys
from collections import Counter
class_counts = Counter(labels)
print(class_counts)



<class 'dict'>
dict_keys(['labels', 'hash', 'results', 'msgs', 'version'])
Counter({23: 1514})


C:\Windows\Temp\ipykernel_22024\3649339855.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels = [int(v['cls']) if isinstance(v['cls'], (int, float)) else int(v['cls'][0]) for v in cache_data['labels']]


In [29]:
import yaml

with open('leaves.yaml', 'r') as f:
    data = yaml.safe_load(f)

print(data)


{'train': 'C:/Users/ojass/Documents/scpr/Leaves/Dataset/train', 'val': 'C:/Users/ojass/Documents/scpr/Leaves/Dataset/val', 'nc': 33, 'names': ['Phyllostachys edulis', 'Aesculus chinensis', 'Berberis anhweiensis', 'Cercis chinensis', 'Indigofera tinctoria', 'Acer Palmatum', 'Phoebe nanmu', 'Kalopanax septemlobus', 'Cinnamomum japonicum', 'Koelreuteria paniculata', 'Ilex macrocarpa', 'Pittosporum tobira', 'Chimonanthus praecox', 'Cinnamomum camphora', 'Viburnum awabuki', 'Osmanthus fragrans', 'Cedrus deodara', 'Ginkgo biloba', 'Lagerstroemia indica', 'Nerium oleander', 'Podocarpus macrophyllus', 'Prunus serrulata', 'Ligustrum lucidum', 'Tonna sinensis', 'Prunus persica', 'Manglietia fordiana', 'Acer buergerianum', 'Mahonia bealei', 'Magnolia grandiflora', 'Populus Ã—canadensis', 'Liriodendron chinense', 'Citrus reticulata', 'Tangerine']}


In [30]:
import os

labels_dir = r'C:\Users\ojass\Documents\scpr\Leaves\Dataset\train\labels'
class_ids = []

for file in os.listdir(labels_dir):
    if file.endswith('.txt'):
        with open(os.path.join(labels_dir, file), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    class_ids.append(int(parts[0]))

from collections import Counter
print(Counter(class_ids))


Counter({23: 1514})


In [12]:
import random
import os
from PIL import Image
import matplotlib.pyplot as plt

# Number of random images to test
num_images = 5

# Path to validation root
val_root = r'C:\Users\ojass\Documents\scpr\Leaves\Dataset\val'

# Walk through subdirectories and collect image paths
image_files = []
for root, dirs, files in os.walk(val_root):
    for file in files:
        if file.lower().endswith(('.jpg', '.png', '.jpeg')):
            image_files.append(os.path.join(root, file))

# Choose random images
random_images = random.sample(image_files, num_images)

# Loop through selected images
for idx, image_path in enumerate(random_images):
    print(f"\nTesting on ({idx+1}/{num_images}): {image_path}")
    
    # Run inference
    results = model(image_path)
    
    # Plot image with bounding boxes
    img_with_boxes = results[0].plot()
    plt.imshow(img_with_boxes)
    plt.axis('off')
    plt.title(f"Prediction: {results[0].names[int(results[0].boxes.cls[0].item())]} "
              f"({results[0].boxes.conf[0].item():.2f})")
    plt.show()
    
    # Print raw prediction data
    print("Class Index:", results[0].boxes.cls)
    print("Confidence :", results[0].boxes.conf)
    print("BBox xyxy  :", results[0].boxes.xyxy)



Testing on (1/5): C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\1101.jpg

image 1/1 C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\1101.jpg: 480x640 1 Tonna sinensis, 20.0ms
Speed: 4.4ms preprocess, 20.0ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>

Class Index: tensor([23.], device='cuda:0')
Confidence : tensor([0.9651], device='cuda:0')
BBox xyxy  : tensor([[1.7090e-01, 1.0035e+00, 1.5995e+03, 1.1994e+03]], device='cuda:0')

Testing on (2/5): C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2034.jpg

image 1/1 C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2034.jpg: 480x640 1 Tonna sinensis, 20.0ms
Speed: 5.1ms preprocess, 20.0ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>

Class Index: tensor([23.], device='cuda:0')
Confidence : tensor([0.9609], device='cuda:0')
BBox xyxy  : tensor([[9.6878e-01, 0.0000e+00, 1.6000e+03, 1.2000e+03]], device='cuda:0')

Testing on (3/5): C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2652.jpg

image 1/1 C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2652.jpg: 480x640 1 Tonna sinensis, 19.8ms
Speed: 6.1ms preprocess, 19.8ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)


...

Class Index: tensor([23.], device='cuda:0')
Confidence : tensor([0.9639], device='cuda:0')
BBox xyxy  : tensor([[3.3524e-01, 3.2410e-01, 1.5994e+03, 1.1997e+03]], device='cuda:0')

Testing on (4/5): C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2582.jpg

image 1/1 C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2582.jpg: 480x640 1 Tonna sinensis, 19.8ms
Speed: 4.6ms preprocess, 19.8ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>

Class Index: tensor([23.], device='cuda:0')
Confidence : tensor([0.9594], device='cuda:0')
BBox xyxy  : tensor([[0.0000e+00, 7.3013e-02, 1.6000e+03, 1.1996e+03]], device='cuda:0')

Testing on (5/5): C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2404.jpg

image 1/1 C:\Users\ojass\Documents\scpr\Leaves\Dataset\val\images\2404.jpg: 480x640 1 Tonna sinensis, 19.7ms
Speed: 5.1ms preprocess, 19.7ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>

Class Index: tensor([23.], device='cuda:0')
Confidence : tensor([0.9699], device='cuda:0')
BBox xyxy  : tensor([[   0.,    0., 1600., 1200.]], device='cuda:0')


In [13]:
import os
from collections import Counter

labels_dir = r"C:\Users\ojass\Documents\scpr\Leaves\Dataset\train\labels"
label_counts = Counter()

for file in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, file), 'r') as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.strip().split()[0])
            label_counts[class_id] += 1

print(label_counts)


Counter({23: 1514})


In [14]:
label_counts = Counter()

for file in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, file), 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if parts:
                class_id = int(parts[0])
                label_counts[class_id] += 1

print(label_counts)


Counter({23: 1514})


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\path\\to\\your\\train\\labels'